In [13]:
## Langchain imports
import langchain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain

## Pinecone imports
import pinecone
from langchain_pinecone import PineconeVectorStore

## OpenAI imports 
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain import OpenAI

## Gemini pro imports

import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [5]:
doc = read_doc('documents/')
doc

[Document(page_content='March 13, 2024\nDear Stripe community:\nCharlie Munger described a two-part rule that works wonders in business, science, and elsewhere:  \n1) take a simple idea and 2) take it very seriously.\nStripe’s mission is to grow the GDP of the internet. The core idea behind the company—one we endeavor \nto take very seriously—is that we’re still early in the journey of software-driven innovation , and Stripe is an \napplied exercise in thinking through some of the corollaries of that. In particular, thanks to the new possi -\nbilities afforded by the internet, we believe that putting better—more global, easier to use, more flexible, \nfaster, cheaper—economic infrastructure in the hands of companies and entrepreneurs will lead to a more \nvibrant and prosperous world. Remarkably, this journey was still in its infancy when Stripe launched in 2011. \nThirteen years, and more than a doubling in internet users later, it’s still early. The world of 2034 is going to \nlook v

In [6]:
#Divide the docs into chunks
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return doc


In [7]:
documents = chunk_data(docs=doc) 
len(documents)

51

In [8]:
# Embedding Technique of OPENAI
embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])

d:\Curious\General-Knowledge-based-app-langchain-pinecone\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [9]:
vectors = embeddings.embed_query("How are you?")
len(vectors)

1536

In [10]:
# Vector search DB in Pinecone
index_name = "langchain-vector"
index = PineconeVectorStore.from_documents(documents, embeddings, index_name=index_name)

In [12]:
# Cosine similarity retreive the results from VectorDB
def retrieve_query(query, k=2):
    matching_results = index.similarity_search(query,k=k)
    return matching_results

In [22]:
llm = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.5)

In [23]:
chain = load_qa_chain(llm, chain_type='stuff')

In [24]:
# Search answers from VectorDB
def retrieve_answers(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents=doc_search,question=query)
    return response


In [25]:
our_query = "What is the advancement in terms of AI/Data that Stripe has done?"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content='tailwind for countless companies across Stripe that do not self-identify as “AI companies” per se.\nWhile headlines inevitably index on the most legible figures, such as declining venture dollars, our net \nassessment is that the startup ecosystem is more vibrant than ever before.\nHow we engineer for reliability\nReliability in the payments industry has historically been a somewhat sorry state of affairs:\n• Payment companies did not adopt the leading high-availability practices of technology companies. \nPayments APIs were flaky and had lots of scheduled (and unscheduled) downtime.\n• Payment companies often made breaking non-backwards compatible API changes with scant warning, \ncausing confusion and downtime for their customers.', metadata={'page': 5.0, 'source': 'documents\\Stripe_2023_annual_letter.pdf'}), Document(page_content='vibrant and prosperous world. Remarkably, this journey was still in its infancy when Stripe launched in 2011. \nThirteen years, a

In [ ]:
from langchain.vectorstores import faiss
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.prompts import PromptTemplate